參考自[https://tf.wiki/zh_hant/basic/basic.html](https://tf.wiki/zh_hant/basic/basic.html)

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11439749727943002330
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4139778048
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15737870039318048557
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [8]:
import tensorflow as tf

* 在TensorFlow 1.X 版本中，必須 在導入TensorFlow套件後呼叫 tf.enable_eager_execution()
* 在TensorFlow 2 中，即時執行模式將成為預設模式，無需額外呼叫 tf.enable_eager_execution() 函數（不過若要關閉即時執行模式，則需呼叫 tf.compat.v1.disable_eager_execution() 函數）

In [12]:
# 定義一個隨機數(純量)
random_float = tf.random.uniform(shape=())
print(random_float)

tf.Tensor(0.4708513, shape=(), dtype=float32)


In [17]:
# 定義一個有2個元素的零向量
zero_vector = tf.zeros(shape=(2), dtype=tf.int32)
print(zero_vector)

tf.Tensor([0 0], shape=(2,), dtype=int32)


In [18]:
# 定義兩個2×2的常量矩陣
A = tf.constant([[1.,2.], [3.,4.]])
B = tf.constant([[5.,6.], [7.,8.]])
print(A, B, sep="\n")

tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[5. 6.]
 [7. 8.]], shape=(2, 2), dtype=float32)


In [19]:
# 查看矩陣A的shape(形狀)、類型和值
print(A.shape)
print(A.dtype)
print(A.numpy())

(2, 2)
<dtype: 'float32'>
[[1. 2.]
 [3. 4.]]


In [21]:
C = tf.add(A, B) #計算矩陣A和B的和
D = tf.matmul(A,B) #計算矩陣A和B的乘積(matrix mul)
print(C)
print(D)

tf.Tensor(
[[ 6.  8.]
 [10. 12.]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[19. 22.]
 [43. 50.]], shape=(2, 2), dtype=float32)


#### TensorFlow 引入了 tf.GradientTape() 這個“推導記錄器” 來實現自動微分
* 以下程式碼為使用 **tf.GradientTape()** 計算函數$y(x)=x^2$在$x=3$時的導數

In [25]:
# x 為一個初始化為3的變數，使用 tf.Variable() 宣告
x = tf.Variable(initial_value=3.)

* 與普通 Tensor 一樣，變數同樣具有形狀、類型和值三種屬性。
* 使用變數需要有一個初始化過程，可以通過在 tf.Variable() 中指定 initial_value 參數來指定初始值。
* 變數與普通 tensor 的一個重要區別是其預設能夠被 TensorFlow 的自動推導機制所求，因此往往被用於定義機器學習模型的參數

In [28]:
# 在 tf.GradientTape() 的上下文內
# 所有計算步驟都會被記錄以用於推導
with tf.GradientTape() as tape:
    y = tf.square(x)
y_grad = tape.gradient(y, x) # 計算y在x的導數(or 求tensor y對變數x的導數)
print(y)
print(y_grad)

tf.Tensor(9.0, shape=(), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)


* 在機器學習中，更加常見的是對多元函數求偏導數，以及對向量或矩陣的推導。
* 以下程式碼展示了如何使用 tf.GradientTape() 計算函數$L(w,b)= {\lVert Xw+b-y\lVert}^2$在$w=(1,2)^T, b=1$時對w,b的偏導數
* $X=\begin{bmatrix}
1 & 2\\
3 & 4
\end{bmatrix},
y=\begin{bmatrix}
1\\
2
\end{bmatrix}$
* 補充
    * ![](https://web.ntnu.edu.tw/~40247038S/Linear%20Algebra/%E7%B7%9A%E6%80%A7%E4%BB%A3%E6%95%B8_files/Image%20[41].png)
    * [線性代數](https://web.ntnu.edu.tw/~40247038S/Linear%20Algebra/)

In [54]:
X = tf.constant([[1., 2.], [3., 4.]])
y = tf.constant([[1.], [2.]])
w = tf.Variable(initial_value=[[1.], [2.]])
b = tf.Variable(initial_value=1.)

In [55]:
with tf.GradientTape() as tape:
    L = tf.reduce_sum(tf.square(tf.matmul(X, w) + b - y))
    # tf.square為矩陣中每個元素做平方
    # tf.reduce_sum 為將矩陣中每個元素相加

In [56]:
print(L)

tf.Tensor(125.0, shape=(), dtype=float32)


In [57]:
# 計算L(w, b)關於w, b的偏導數
w_grad, b_grad = tape.gradient(L, [w, b])
print(L, w_grad, b_grad)

tf.Tensor(125.0, shape=(), dtype=float32) tf.Tensor(
[[ 70.]
 [100.]], shape=(2, 1), dtype=float32) tf.Tensor(30.0, shape=(), dtype=float32)


![](https://tf.wiki/_images/math/1ac464fd70b56c10c245f81d76eaf97e7e436d54.png)

### Linear Regression

In [58]:
import numpy as np

* 求 y = a x + b

|年份|2013|2014|2015|2016|2017|
|----|----|----|----|----|----|
|房價|12000|14000|15000|16500|17500|

In [61]:
X_raw = np.array([2013, 2014, 2015, 2016, 2017], dtype=np.float32)
y_raw = np.array([12000, 14000, 15000, 16500, 17500], dtype=np.float32)
print(X_raw)
print(y_raw)

[2013. 2014. 2015. 2016. 2017.]
[12000. 14000. 15000. 16500. 17500.]


In [63]:
# 正規化
X = (X_raw - X_raw.min()) / (X_raw.max() - X_raw.min())
print(X)
y = (y_raw - y_raw.min()) / (y_raw.max() - y_raw.min())
print(y)

[0.   0.25 0.5  0.75 1.  ]
[0.         0.36363637 0.54545456 0.8181818  1.        ]


In [71]:
a, b = 0, 0 #初始化

num_epoch = 100000
learning_rate = 5e-4

for e in range(num_epoch):
    # 手動計算損失函數對自變數（模型參數）的梯度
    y_predict = a * X + b
    # loss = (y_predict - y)**2
    # 以下手動對a做偏微分
    grad_a = 2 * (y_predict - y).dot(X)
    # 以下手動對b作偏微分
    grad_b = 2 * (y_predict - y).sum()
    # update a,b
    a, b = a - learning_rate * grad_a, b- learning_rate * grad_b
    


print(a, b)

0.9818181692582533 0.05454546396809918


#### TensorFlow 下的線性回歸

In [72]:
X = tf.constant(X)
y = tf.constant(y)

In [73]:
a = tf.Variable(initial_value=0.)
b = tf.Variable(initial_value=0.)
variables = [a, b]

In [78]:
num_epoch = 100000
optimizer = tf.keras.optimizers.SGD(learning_rate=5e-4)
for e in range(num_epoch):
    # 使用tf.GradientTape()記錄損失函數的梯度資訊
    with tf.GradientTape() as tape:
        y_pred = a * X + b
        loss = tf.reduce_sum(tf.square(y_pred - y))
    # TensorFlow自動計算損失函數關於自變數（模型參數）的梯度
    grads = tape.gradient(loss, variables)
    optimizer.apply_gradients(grads_and_vars = zip(grads, variables))
    
print(a, b)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.98176914> <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.054570343>
